In [5]:
from seg_xcoders import *
import argparse
from transformers import AutoTokenizer
import torch
import os
from pointllm.conversation import conv_templates, SeparatorStyle
from pointllm.utils import disable_torch_init
from pointllm.model import *
from pointllm.model.utils import KeywordsStoppingCriteria

from pointllm.data import load_ulip2_objaverse_point_cloud

import os

In [6]:
def init_model(args):
    # Model
    disable_torch_init()

    model_path = args.model_path 
    print(f'[INFO] Model name: {model_path}')

    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = PointLLMLlamaForCausalLM.from_pretrained(model_path, low_cpu_mem_usage=False, use_cache=True, torch_dtype=torch.float16).cuda()
                                                     # args.torch_dtype).cuda()
    model.initialize_tokenizer_point_backbone_config_wo_embedding(tokenizer)

    model.eval()

    mm_use_point_start_end = getattr(model.config, "mm_use_point_start_end", False)
    # Add special tokens ind to model.point_config
    point_backbone_config = model.get_model().point_backbone_config
    
    if mm_use_point_start_end:
        if "v1" in model_path.lower():
            conv_mode = "vicuna_v1_1"
        else:
            raise NotImplementedError

        conv = conv_templates[conv_mode].copy()

    stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
    keywords = [stop_str]
    
    return model, tokenizer, point_backbone_config, keywords, mm_use_point_start_end, conv

In [7]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--model-path", type=str, \
default="RunsenXu/PointLLM_7B_v1.1")

parser.add_argument("--data-path", type=str, default="/data2/llf/objaverse/8192_npy")
parser.add_argument("--torch-dtype", type=str, default="float32", choices=["float32", "float16", "bfloat16"])
args = parser.parse_args(args=[])
    # '--torch-dtype','float32'])
model, tokenizer, point_backbone_config, keywords, mm_use_point_start_end, conv = init_model(args)

[INFO] Model name: RunsenXu/PointLLM_7B_v1.1
Loading PointBERT config from /home/linfeng/PointLLM/pointllm/model/pointbert/PointTransformer_base_8192point.yaml.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
point_backbone_config

{'point_cloud_dim': 6,
 'backbone_output_dim': 1152,
 'project_output_dim': 4096,
 'point_token_len': 513,
 'mm_use_point_start_end': True,
 'projection_hidden_layer': 0,
 'use_max_pool': False,
 'default_point_patch_token': '<point_patch>',
 'point_patch_token': 32000,
 'default_point_start_token': '<point_start>',
 'default_point_end_token': '<point_end>',
 'point_start_token': 32001,
 'point_end_token': 32002}